In [15]:
library(geojsonio)
library(tidyverse)
library(dplyr)
library("knitr")
library(units)
library(sf)

Linking to GEOS 3.12.1, GDAL 3.8.4, PROJ 9.3.1; sf_use_s2() is TRUE



In [45]:
# Read in the data
census_index_crime <- st_read("census_index_crime.geojson") |>
    mutate(across(c(immigrant_prop, education_prop, mean_land_value,income), as.double))|>
    na.omit()|>
    glimpse()



Reading layer `census_index_crime' from data source 
  `C:\Users\Kaiyan Zhang\Desktop\econ326\census_index_crime.geojson' 
  using driver `GeoJSON'
Simple feature collection with 993 features and 17 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -123.2242 ymin: 49.19853 xmax: -123.0229 ymax: 49.31408
Geodetic CRS:  WGS 84


Warning message:
"There were 4 warnings in `stopifnot()`.
The first warning was:
ℹ In argument: `across(...)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 3 remaining warnings."


Rows: 989
Columns: 18
$ name               <chr> "59150307", "59150308", "59150309", "59150310", "59…
$ households         <int> 254, 203, 269, 283, 181, 203, 402, 170, 333, 174, 2…
$ dwellings          <int> 273, 223, 299, 310, 199, 211, 418, 181, 356, 189, 3…
$ population         <int> 632, 501, 745, 536, 532, 562, 1088, 556, 959, 506, …
$ population_density <dbl> 2110.9, 4575.3, 6663.7, 4895.0, 6567.9, 6445.0, 629…
$ region             <chr> "Vancouver", "Vancouver", "Vancouver", "Vancouver",…
$ area_sq_km         <dbl> 0.29926, 0.10956, 0.11190, 0.10940, 0.08094, 0.0870…
$ age                <int> 630, 500, 745, 540, 530, 560, 1085, 555, 960, 510, …
$ income             <dbl> 0.90965, 0.93611, 0.84736, 0.45184, 0.82517, 0.7436…
$ immigrant_prop     <dbl> 0.2610759, 0.2894212, 0.3557047, 0.2891791, 0.40413…
$ education_prop     <dbl> 0.3085443, 0.2794411, 0.3154362, 0.1865672, 0.25375…
$ school_index       <dbl> 1.074675, 1.252273, 1.426336, 2.441091, 1.543095, 1…
$ light_index     

In [46]:
regression_simple <- lm(crime_rate ~ school_index + light_index + bus_index + immigrant_prop + education_prop, data = census_index_crime)|>
    summary()
    
regression_simple


Call:
lm(formula = crime_rate ~ school_index + light_index + bus_index + 
    immigrant_prop + education_prop, data = census_index_crime)

Residuals:
    Min      1Q  Median      3Q     Max 
-2111.9  -307.2   -95.9   121.8 12837.1 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -502.44     166.85  -3.011  0.00267 ** 
school_index    -290.68      36.85  -7.887 8.20e-15 ***
light_index      434.14      43.59   9.958  < 2e-16 ***
bus_index        112.60      50.45   2.232  0.02584 *  
immigrant_prop  -689.38     174.33  -3.954 8.22e-05 ***
education_prop   114.32     193.12   0.592  0.55400    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 743.5 on 983 degrees of freedom
Multiple R-squared:  0.2241,	Adjusted R-squared:  0.2202 
F-statistic:  56.8 on 5 and 983 DF,  p-value: < 2.2e-16


In [47]:
reg_simple <- lm(crime_rate ~ mean_land_value + school_index + light_index + bus_index + immigrant_prop + education_prop, data = census_index_crime)
    
y_hat <- predict(reg_simple)

length(y_hat)

[1] 989

In [49]:
white <- lm(census_index_crime$crime_rate - y_hat ~ y_hat + I(y_hat^2))|>
    summary()

white


Call:
lm(formula = census_index_crime$crime_rate - y_hat ~ y_hat + 
    I(y_hat^2))

Residuals:
    Min      1Q  Median      3Q     Max 
-4771.2  -167.3   -46.4    80.5 10520.9 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.928e+02  3.421e+01   5.636 2.27e-08 ***
y_hat       -1.235e+00  9.351e-02 -13.204  < 2e-16 ***
I(y_hat^2)   1.008e-03  6.323e-05  15.946  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 660.8 on 986 degrees of freedom
Multiple R-squared:  0.205,	Adjusted R-squared:  0.2034 
F-statistic: 127.1 on 2 and 986 DF,  p-value: < 2.2e-16
